In [ ]:
import numpy as np
import pygetm

%matplotlib widget
import matplotlib.pyplot

## Set up the domain (Cartesian box)

In [ ]:
Lx, Ly = 100e3, 100e3
nx, ny = 100, 52
Am = 10.0

x = np.linspace(-Lx / 2, Lx / 2, nx)
y = np.linspace(-Ly / 2, Ly / 2, ny)
domain = pygetm.domain.create_cartesian(x, y, H=10.0, lat=0.0)
T = domain.create_grids(1, halox=2, haloy=2, velocity_grids=2)
U = T.ugrid
V = T.vgrid

## Dirac delta in x-direction (not physical!)

In [ ]:
u = U.array(fill=0.0)
v = V.array(fill=0.0)
icenter = nx // 2 - 1
u[:, icenter] = 1.0

fig, ax = matplotlib.pyplot.subplots()
(model_line,) = ax.plot(u.grid.x[0, :], u[0, :], label="numerical")
ax.grid()
ax.set_ylabel("distance (m)")
ax.set_ylabel("u velocity (m s-1)")

## Animate

In [ ]:
timestep = 600.0
ntime = 1000
tol = 1e-15
diffU = U.array(fill=0.0)
diffV = V.array(fill=0.0)

xa = U.x.values[0, :] - U.x.values[0, icenter]
(analytical_line,) = ax.plot(u.grid.x[0, :], u[0, :], label="analytical")
ax.legend()

for itime in range(ntime):
    pygetm._pygetm.momentum_diffusion(
        u.grid.ugrid.D,
        u.grid.vgrid.D,
        v.grid.ugrid.D,
        v.grid.vgrid.D,
        u,
        v,
        Am,
        diffU,
        diffV,
    )
    u.all_values += diffU.all_values / u.grid.D.all_values * timestep
    v.all_values += diffV.all_values / v.grid.D.all_values * timestep
    model_line.set_ydata(u[0, :])
    va = 2 * (2 * Am * itime * timestep)
    analytical = (
        1.0 / np.sqrt(2 * np.pi * va) * np.exp(-0.5 * xa**2 / va) * U.dx.values[0, :]
    )
    analytical_line.set_ydata(analytical)
    fig.canvas.draw()
assert (np.abs(u.ma - u.ma[0, :]) < tol).all()
assert (np.abs(v.ma) < tol).all()

# Dirac delta in y-direction (not physical!)

In [ ]:
u = U.array(fill=0.0)
v = V.array(fill=0.0)
jcenter = ny // 2 - 1
v[jcenter, :] = 1.0

fig, ax = matplotlib.pyplot.subplots()
(model_line,) = ax.plot(v.grid.y[:, 0], v[:, 0], label="numerical")
ax.grid()
ax.set_ylabel("distance (m)")
ax.set_ylabel("v velocity (m s-1)")

## Animate

In [ ]:
timestep = 600.0
ntime = 1000
tol = 1e-15
diffU = U.array(fill=0.0)
diffV = V.array(fill=0.0)

ya = V.y.values[:, 0] - V.y.values[jcenter, 0]
(analytical_line,) = ax.plot(v.grid.y[:, 0], v[:, 0], label="analytical")
ax.legend()

for itime in range(ntime):
    pygetm._pygetm.momentum_diffusion(
        u.grid.ugrid.D,
        u.grid.vgrid.D,
        v.grid.ugrid.D,
        v.grid.vgrid.D,
        u,
        v,
        Am,
        diffU,
        diffV,
    )
    u.all_values += diffU.all_values / u.grid.D.all_values * timestep
    v.all_values += diffV.all_values / v.grid.D.all_values * timestep
    model_line.set_ydata(v[:, 0])
    va = 2 * (2 * Am * itime * timestep)
    analytical = (
        1.0 / np.sqrt(2 * np.pi * va) * np.exp(-0.5 * ya**2 / va) * V.dy.values[:, 0]
    )
    analytical_line.set_ydata(analytical)
    fig.canvas.draw()

assert (np.abs(v.ma - v.ma[:, 0, np.newaxis]) < tol).all()
assert (np.abs(u.ma) < tol).all()